In [1]:
!brew install deno

==> Auto-updating Homebrew...
Adjust how often this is run with `$HOMEBREW_AUTO_UPDATE_SECS` or disable with
`$HOMEBREW_NO_AUTO_UPDATE=1`. Hide these hints with `$HOMEBREW_NO_ENV_HINTS=1` (see `man brew`).
==> Auto-updated Homebrew!
Updated 2 taps (homebrew/core and homebrew/cask).
==> New Formulae
airtable-mcp-server: MCP Server for Airtable
cagent: Agent Builder and Runtime by Docker Engineering
chrome-devtools-mcp: Chrome DevTools for coding agents
fernflower: Advanced decompiler for Java bytecode
jiratui: Textual User Interface for interacting with Atlassian Jira from your shell
mcp-google-sheets: MCP server integrates with your Google Drive and Google Sheets
mcp-grafana: MCP server for Grafana
mdserve: Fast markdown preview server with live reload and theme support
mlx-lm: Run LLMs with MLX
openssl@3.5: Cryptography and SSL/TLS Toolkit
playwright-mcp: MCP server for Playwright
portable-libffi: Portable Foreign Function Interface library
portable-libxcrypt: Extended crypt library f

In [2]:
!deno install

Download ▰▰▱▱▱▱ [00:00] 1/6
  npm:pyodide/-/pyodide-0.26.2.tgz
Download ▰▰▰▱▱▱ [00:00] 1/6
  npm:pyodide/-/pyodide-0.26.2.tgz
Download ▰▰▰▰▱▱ [00:00] 3/7
  npm:pyodide/-/pyodide-0.26.2.tgz 0.40MiB/5.59MiB
Download ▰▰▰▰▰▱ [00:00] 4/7
  npm:pyodide/-/pyodide-0.26.2.tgz 1.63MiB/5.59MiB
Download ▰▰▰▰▰▰ [00:00] 4/7
  npm:pyodide/-/pyodide-0.26.2.tgz 5.43MiB/5.59MiB
Installed 2 packages in 704ms
Reused 0 packages from cache
Downloaded 0 packages from JSR
Downloaded 4 packages from npm
++++

Dependencies:
+ npm:pyodide 0.26.2



In [9]:
%pip install dspy langchain langchain-community yfinance

Note: you may need to restart the kernel to use updated packages.


In [10]:
# DSPy configuration for Financial Analyst agent
import os
import dspy
import yfinance as yf

# Optionally load environment from .env if available
try:
    from dotenv import load_dotenv  # type: ignore
    load_dotenv()
except Exception:
    pass

# Configure model provider (OpenAI-only, consistent with other agents)
lm = dspy.LM("openai/gpt-5-mini", api_key=os.getenv("OPENAI_API_KEY"), temperature=1, max_tokens=16000)

dspy.configure(lm=lm)

print("DSPy configured for Financial Analyst agent.")


DSPy configured for Financial Analyst agent.


In [11]:
# Configure Deno for DSPy ProgramOfThought (use workspace deno.json)
import os, shutil, pathlib

deno_cfg = str((pathlib.Path.cwd() / "deno.json").resolve())
os.environ["DENO_CONFIG"] = deno_cfg

if shutil.which("deno") is None:
    for p in ["/opt/homebrew/bin", "/usr/local/bin"]:
        if p not in os.environ.get("PATH", ""):
            os.environ["PATH"] = os.environ.get("PATH", "") + f":{p}"

print("DENO_CONFIG:", os.environ.get("DENO_CONFIG"))
print("deno in PATH:", shutil.which("deno"))


DENO_CONFIG: /Users/hammer-mt/Documents/Codes/ai-agents-udemy-course/dspy/financial_analyst_agent/deno.json
deno in PATH: /opt/homebrew/bin/deno


In [6]:
# Yahoo Finance tools (simple, deterministic wrappers)
from typing import List, Optional, Literal, Dict, Any


def yf_get_price(ticker: str) -> str:
    """Return latest price info for a ticker as a compact string.
    Designed for tool use within ReAct.
    """
    if not ticker or not isinstance(ticker, str):
        return "Invalid ticker"
    try:
        t = yf.Ticker(ticker)
        info = t.fast_info if hasattr(t, "fast_info") else {}
        last = getattr(info, "last_price", None) if hasattr(info, "last_price") else info.get("last_price") if isinstance(info, dict) else None
        currency = getattr(info, "currency", None) if hasattr(info, "currency") else info.get("currency") if isinstance(info, dict) else None
        if last is None:
            data = t.history(period="1d")
            if data is None or data.empty:
                return f"No price data for {ticker}."
            last = float(data["Close"].iloc[-1])
        return f"{ticker} price: {last} {currency or ''}".strip()
    except Exception as e:
        return f"Error fetching price for {ticker}: {e}"


def yf_get_history(ticker: str, period: str = "5d", interval: str = "1d") -> str:
    """Return recent historical OHLCV rows in a compact, newline-separated string.
    Keep output concise for model consumption in ReAct.
    """
    if not ticker:
        return "Invalid ticker"
    try:
        # Use Ticker().history to avoid MultiIndex columns from download()
        t = yf.Ticker(ticker)
        data = t.history(period=period, interval=interval, auto_adjust=False)
        if data is None or data.empty:
            return f"No history for {ticker}."
        # Keep last up to 10 rows for brevity
        tail = data.tail(10)
        lines = []
        for idx, row in tail.iterrows():
            try:
                o = float(row["Open"]) if row.get("Open") is not None else None
                h = float(row["High"]) if row.get("High") is not None else None
                l = float(row["Low"]) if row.get("Low") is not None else None
                c = float(row["Close"]) if row.get("Close") is not None else None
                v_raw = row.get("Volume")
                v = int(v_raw) if v_raw is not None and not (hasattr(v_raw, "isna") and v_raw.isna()) else 0
                o_str = f"{o:.2f}" if o is not None else ""
                h_str = f"{h:.2f}" if h is not None else ""
                l_str = f"{l:.2f}" if l is not None else ""
                c_str = f"{c:.2f}" if c is not None else ""
                v_str = str(v)
            except Exception:
                o_str = str(row.get("Open", ""))
                h_str = str(row.get("High", ""))
                l_str = str(row.get("Low", ""))
                c_str = str(row.get("Close", ""))
                v_str = str(row.get("Volume", ""))
            lines.append(f"{idx.date()} O={o_str} H={h_str} L={l_str} C={c_str} V={v_str}")
        return f"History {ticker} ({period}, {interval}):\n" + "\n".join(lines)
    except Exception as e:
        return f"Error fetching history for {ticker}: {e}"


In [12]:
hist_aapl = yf_get_history("AAPL", period="5d", interval="1d")
print(hist_aapl)

History AAPL (5d, 1d):
2025-09-29 O=254.56 H=255.00 L=253.01 C=254.43 V=40127700
2025-09-30 O=254.86 H=255.92 L=253.11 C=254.63 V=37704300
2025-10-01 O=255.04 H=258.79 L=254.93 C=255.45 V=48713900
2025-10-02 O=256.58 H=258.18 L=254.15 C=257.13 V=42630200
2025-10-03 O=254.67 H=259.24 L=253.95 C=258.02 V=49107000


In [ ]:
# ProgramOfThought: analysis signature + module
# Reference: DSPy ProgramOfThought tutorial
# https://dspy.ai/tutorials/program_of_thought/?h=programofthought#tutorial-programofthought
# https://dspy.ai/tutorials/yahoo_finance_react/?h=yahoo

class FinanceAnalysis(dspy.Signature):
    """Write Python code to analyze provided financial data/questions and output the final numeric or textual result.

    Keep code minimal and safe; print only the final result.
    """
    question = dspy.InputField()
    context = dspy.InputField()
    answer = dspy.OutputField()


# Create the ProgramOfThought program for finance analysis
finance_pot = dspy.ProgramOfThought(FinanceAnalysis)


def run_finance_analysis(question: str, context: str = "") -> str:
    """Helper wrapper to run ProgramOfThought for finance tasks."""
    try:
        pred = finance_pot(question=question, context=context)
        return getattr(pred, "answer", "")
    except Exception as e:
        return f"PoT error: {e}"



In [14]:
run_finance_analysis(question="What is the 5-day return for AAPL?", context=hist_aapl)

'5-day return = 0.01410997131 (≈ 1.410997131%, ≈ 1.41%)'

In [15]:
lm.inspect_history(n=1)





[2025-10-04T13:35:40.524368]

System message:

Your input fields are:
1. `question` (str): 
2. `context` (str): 
3. `final_generated_code` (str): python code that answers the question
4. `code_output` (str): output of previously-generated python code
Your output fields are:
1. `reasoning` (str): 
2. `answer` (str):
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## context ## ]]
{context}

[[ ## final_generated_code ## ]]
{final_generated_code}

[[ ## code_output ## ]]
{code_output}

[[ ## reasoning ## ]]
{reasoning}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Given the final code `question`, `context`, `final_generated_code`, `code_output`, provide the final `answer`.


User message:

[[ ## question ## ]]
What is the 5-day return for AAPL?

[[ ## context ## ]]
History AAPL (5d, 1d):
2025-09-29 O=254.56 H=255.00 L=253.01 C=254.43 V

In [17]:
# ReAct signature for Financial Analysis

class FinanceReActSignature(dspy.Signature):
    """
    You are a financial analyst. Use tools to fetch market data and perform deterministic analyses.

    Tools available:
    - yf_get_price(ticker: str) -> str  # latest price summary
    - yf_get_history(ticker: str, period: str = "5d", interval: str = "1d") -> str  # recent OHLCV
    - run_finance_analysis(question: str, context: str = "") -> str  # ProgramOfThought analysis

    Behavior:
    - For data retrieval, call yf_get_price / yf_get_history.
    - For calculations, comparisons, or aggregations, call run_finance_analysis with a concise question and include retrieved context.
    - Finish with a concise, well-structured answer.

    Outputs:
    - action: primary tool used (one of: yf_get_price, yf_get_history, run_finance_analysis, answer_direct)
    - tool_result: the most relevant tool output used
    - answer: final financial analysis answer
    """
    user_message: str = dspy.InputField(description="User's finance request")
    history: dspy.History = dspy.InputField(description="Conversation history")

    reasoning: str = dspy.OutputField(description="Brief plan of tool calls and why")
    action: str = dspy.OutputField(description="Chosen primary action/tool")
    tool_result: str = dspy.OutputField(description="Tool output used")
    answer: str = dspy.OutputField(description="Final answer")


In [19]:
# FinancialAnalystAgent module using ReAct + ProgramOfThought

class FinancialAnalystAgent(dspy.Module):
    def __init__(self, max_iters: int = 5):
        super().__init__()
        self.conversation_history = dspy.History(messages=[])
        self.react = dspy.ReAct(
            FinanceReActSignature,
            tools=[yf_get_price, yf_get_history, run_finance_analysis],
            max_iters=max_iters,
        )

    def forward(self, user_message: str):
        # Append user message to internal history
        self.conversation_history.messages.append({"role": "user", "content": user_message})
        # Run ReAct with internal history
        result = self.react(user_message=user_message, history=self.conversation_history)
        # Capture assistant response back to history
        answer = getattr(result, "answer", "")
        if isinstance(answer, str) and answer.strip():
            self.conversation_history.messages.append({"role": "assistant", "content": answer})
        return result


In [20]:
# Instantiate the Financial Analyst agent
finance_agent = FinancialAnalystAgent(max_iters=5)
print("FinancialAnalystAgent ready.")


FinancialAnalystAgent ready.


In [21]:
# Example: Compare 5-day returns for two tickers using ReAct tools + ProgramOfThought

# Step 1: Retrieve recent history for two tickers (e.g., AAPL and MSFT)
hist_aapl = yf_get_history("AAPL", period="5d", interval="1d")
hist_msft = yf_get_history("MSFT", period="5d", interval="1d")

print(hist_aapl.splitlines()[0])
print(hist_aapl)
print()
print(hist_msft.splitlines()[0])
print(hist_msft)

# Step 2: Ask ProgramOfThought to compute 5-day percent returns and compare
context = f"AAPL history:\n{hist_aapl}\n\nMSFT history:\n{hist_msft}"
question = (
    "Using the provided daily Close prices for AAPL and MSFT, compute each ticker's 5-day return in percent "
    "(from oldest Close to newest Close), round to two decimals, and then state which outperformed."
)

comparison_result = run_finance_analysis(question=question, context=context)
print("\nPoT comparison result:\n", comparison_result)

# Optional: show a ReAct chat example using the agent (one shot)
resp = finance_agent("Compare AAPL and MSFT 5-day performance and provide the numeric returns.")
print({
    "answer": getattr(resp, "answer", ""),
    "action": getattr(resp, "action", ""),
    "tool_result": getattr(resp, "tool_result", "")[:500],
})


History AAPL (5d, 1d):
History AAPL (5d, 1d):
2025-09-29 O=254.56 H=255.00 L=253.01 C=254.43 V=40127700
2025-09-30 O=254.86 H=255.92 L=253.11 C=254.63 V=37704300
2025-10-01 O=255.04 H=258.79 L=254.93 C=255.45 V=48713900
2025-10-02 O=256.58 H=258.18 L=254.15 C=257.13 V=42630200
2025-10-03 O=254.67 H=259.24 L=253.95 C=258.02 V=49107000

History MSFT (5d, 1d):
History MSFT (5d, 1d):
2025-09-29 O=511.50 H=516.85 L=508.88 C=514.60 V=17617800
2025-09-30 O=513.24 H=518.16 L=509.66 C=517.95 V=19728200
2025-10-01 O=514.80 H=520.51 L=511.69 C=519.71 V=22632300
2025-10-02 O=517.64 H=521.60 L=510.68 C=515.74 V=21222900
2025-10-03 O=517.10 H=520.49 L=515.00 C=517.35 V=15104200

PoT comparison result:
 AAPL 5-day return: 1.41%
MSFT 5-day return: 0.53%
Outperformed: AAPL
{'answer': '5-day returns (first close -> last close):\n- AAPL: 254.43 -> 258.02 = +3.59 (+1.41%)\n- MSFT: 514.60 -> 517.35 = +2.75 (+0.53%)\n\nOutperformer: AAPL (higher 5-day percent return)', 'action': 'run_finance_analysis', 'too

In [22]:
resp.trajectory

{'thought_0': "I'll fetch the 5-day daily OHLCV history for AAPL first, then I'll fetch MSFT and calculate the numeric 5-day returns for each.",
 'tool_name_0': 'yf_get_history',
 'tool_args_0': {'ticker': 'AAPL', 'period': '5d', 'interval': '1d'},
 'observation_0': 'History AAPL (5d, 1d):\n2025-09-29 O=254.56 H=255.00 L=253.01 C=254.43 V=40127700\n2025-09-30 O=254.86 H=255.92 L=253.11 C=254.63 V=37704300\n2025-10-01 O=255.04 H=258.79 L=254.93 C=255.45 V=48713900\n2025-10-02 O=256.58 H=258.18 L=254.15 C=257.13 V=42630200\n2025-10-03 O=254.67 H=259.24 L=253.95 C=258.02 V=49107000',
 'thought_1': 'Now fetch MSFT 5-day daily OHLCV history so I can compute and compare the 5-day numeric returns for AAPL vs MSFT.',
 'tool_name_1': 'yf_get_history',
 'tool_args_1': {'ticker': 'MSFT', 'period': '5d', 'interval': '1d'},
 'observation_1': 'History MSFT (5d, 1d):\n2025-09-29 O=511.50 H=516.85 L=508.88 C=514.60 V=17617800\n2025-09-30 O=513.24 H=518.16 L=509.66 C=517.95 V=19728200\n2025-10-01 O=514

In [23]:
lm.inspect_history(n=1)





[2025-10-04T13:39:55.385815]

System message:

Your input fields are:
1. `user_message` (str): User's finance request
2. `history` (History): Conversation history
3. `trajectory` (str):
Your output fields are:
1. `reasoning` (str): Brief plan of tool calls and why
2. `action` (str): Chosen primary action/tool
3. `tool_result` (str): Tool output used
4. `answer` (str): Final answer
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## user_message ## ]]
{user_message}

[[ ## history ## ]]
{history}

[[ ## trajectory ## ]]
{trajectory}

[[ ## reasoning ## ]]
{reasoning}

[[ ## action ## ]]
{action}

[[ ## tool_result ## ]]
{tool_result}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        You are a financial analyst. Use tools to fetch market data and perform deterministic analyses.
        
        Tools available:
        - yf_get_price(ticker: str) -> str  # latest price sum